In [ ]:
import os
import zipfile

# Descargar dataset desde Kaggle
dataset_path = "alzheimers_data"
kaggle_dataset = "yiweilu2033/well-documented-alzheimers-dataset"

# Crear la carpeta si no existe
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path, exist_ok=True)
    os.system(f"kaggle datasets download -d {kaggle_dataset} -p {dataset_path} --unzip")
    print("Dataset descargado y descomprimido en:", dataset_path)
else:
    print("El dataset ya está descargado.")


Dataset descargado y descomprimido en: alzheimers_data


In [ ]:
import os


# Verificar si la ruta del dataset existe
if not os.path.exists(dataset_path):
    print("Error: La ruta del dataset no existe.")
else:
    # Diccionario para almacenar el número de imágenes por clase
    image_counts = {}

    # Recorrer las carpetas dentro del dataset
    for class_name in sorted(os.listdir(dataset_path)):  # Ordenar alfabéticamente
        class_path = os.path.join(dataset_path, class_name)

        if os.path.isdir(class_path):  # Verificar que es una carpeta
            num_images = 0

            # Recorrer todos los archivos en la carpeta y sus subcarpetas
            for root, _, files in os.walk(class_path):
                num_images += sum(1 for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')))

            image_counts[class_name] = num_images

    # Mostrar el número de imágenes por clase
    for class_name, count in image_counts.items():
        print(f" {class_name}: {count} imágenes")

    # Mostrar total de imágenes en todo el dataset
    total_images = sum(image_counts.values())
    print(f"\n Total de imágenes en el dataset: {total_images}")


 MildDemented: 5184 imágenes
 ModerateDemented: 376 imágenes
 NonDemented (2): 63560 imágenes
 VeryMildDemented: 13796 imágenes

 Total de imágenes en el dataset: 82916


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformaciones para preprocesar imágenes (según el modelo del paper)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ajuste inicial según la entrada del modelo
    transforms.RandomHorizontalFlip(p=0.5),  # Aumentación de datos
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalización en imágenes RGB
])

# Cargar dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# División en 80%-10%-10%
train_size = int(0.80 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size  # Ajuste para que sumen correctamente

torch.manual_seed(42)  # Reproducibilidad
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Crear dataloaders
batch_size = 128  # Reducido para evitar problemas de memoria
num_workers = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 66332, Validation: 8291, Test: 8293


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlzheimerCNN(nn.Module):
    def __init__(self, num_classes=1):  # El paper usa salida binaria
        super(AlzheimerCNN, self).__init__()

        # Bloque 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bloque 2
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bloque 3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bloque 4
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bloque 5
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calcular tamaño de la salida antes de la capa fully connected
        self.flattened_size = self._get_flattened_size()

        # Capas totalmente conectadas
        self.fc1 = nn.Linear(self.flattened_size, 121)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(121, num_classes)

    def _get_flattened_size(self):
        """Calcula automáticamente el tamaño de la salida antes de la capa fully connected."""
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 224, 224)  # Imagen de prueba con tamaño 224x224
            x = self.pool1(F.relu(self.conv1(dummy_input)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.pool3(F.relu(self.conv3(x)))
            x = self.pool4(F.relu(self.conv4(x)))
            x = self.pool5(F.relu(self.conv5(x)))
            return x.view(1, -1).size(1)  # Obtener tamaño correcto para la capa FC

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.pool5(F.relu(self.conv5(x)))

        x = torch.flatten(x, 1)  # Aplanar para fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Inicializar modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlzheimerCNN(num_classes=4).to(device)

# Comprobar arquitectura
print(model)

# Prueba con un batch de imágenes de 224x224
dummy_images = torch.randn(16, 3, 224, 224).to(device)
outputs = model(dummy_images)

print("Output shape:", outputs.shape)  # Debería ser [16, 4] porque hay 4 clases


AlzheimerCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=121, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=121, out_features=4, bias=True)
)
Outp

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import time

# Configuración de hiperparámetros
criterion = torch.nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizador Adam
num_epochs = 25  # Número de épocas

# Entrenamiento del modelo
start_time = time.time()

for epoch in range(num_epochs):
    model.train()  # Modo de entrenamiento
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Reiniciar gradientes
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Cálculo de pérdida
        loss.backward()  # Backpropagation
        optimizer.step()  # Actualizar pesos

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total

    # Evaluación en validación
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Loss: {running_loss/len(train_loader):.4f}, "
          f"Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {val_accuracy:.2f}%")

training_time = time.time() - start_time
print(f"\nTiempo total de entrenamiento: {training_time:.2f} segundos")

# Guardar modelo entrenado
torch.save(model.state_dict(), "alzheimers_model.pth")
print("Modelo guardado como 'alzheimers_model.pth'")


Epoch [1/25] - Loss: 0.6299, Accuracy: 76.38%, Val Loss: 0.5847, Val Accuracy: 76.50%
Epoch [2/25] - Loss: 0.5312, Accuracy: 77.17%, Val Loss: 0.4707, Val Accuracy: 77.54%


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Evaluación en el conjunto de test
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Matriz de confusión
conf_matrix = confusion_matrix(all_labels, all_preds)

# Visualización
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Reporte de clasificación
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=dataset.classes))


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Binarización de etiquetas para ROC (One-vs-Rest)
num_classes = len(dataset.classes)
y_true_bin = label_binarize(all_labels, classes=range(num_classes))
y_pred_prob = torch.softmax(torch.tensor(all_preds), dim=1).cpu().numpy()  # Convertimos las predicciones a probabilidades

# Curva ROC para cada clase
plt.figure(figsize=(8,6))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {dataset.classes[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')  # Línea base
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves")
plt.legend(loc="lower right")
plt.show()
